# PATATO: A Python Photoacoustic Tomography Analysis Toolkit

In [1]:
import patato as pat
from patato.data import get_msot_time_series_example, get_msot_phantom_example
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pa_so2 = get_msot_time_series_example("so2")
pa_dce = get_msot_time_series_example("icg") # We'll get the ROIs from here.
pa_so2.set_default_recon(("Reference Backprojection", "0"))
pa_dce.set_default_recon(("Reference Backprojection", "0"))
pa_dce.default_unmixing_type = "ICG"
pa_dce.external_roi_interface = pa_so2

# Get the oe reconstructions, so2 and delta icgs:
rec = pa_so2.get_scan_reconstructions()
so2 = pa_so2.get_scan_so2()
icg = pa_dce.get_scan_unmixed()[:, 2:] # ICG is the 3rd dataset in the unmixed data.

times = pa_so2.get_timestamps()[:, 0]
times -= times[0]
times /= 60

icg_times = pa_dce.get_timestamps()[:, 0]
icg_times -= icg_times[0]
icg_times /= 60

roi_tumour_right = pa_so2.get_rois()["tumour_right", "0"]
roi_tumour_left = pa_so2.get_rois()["tumour_left", "0"]
roi_reference = pa_so2.get_rois()["reference_", "0"]

/home/cri.camres.org/else01/patato/patato/core/image_structures/reconstruction_image.py:43: UserWarning: Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.
  warnings.warn("Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.")
/home/cri.camres.org/else01/patato/patato/core/image_structures/reconstruction_image.py:43: UserWarning: Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.
  warnings.warn("Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.")
/home/cri.camres.org/else01/patato/patato/core/image_structures/reconstruction_image.py:43: UserWarning: Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.
  warnings.warn("Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.")


In [3]:
def get_trace(time_series_data, region, filter_bad=False):
    mask, _ = region.to_mask_slice(time_series_data)
    timeseries = time_series_data.raw_data.T[mask.T].T[:, 0, :]
    # Unmixed SO2 can be out of the interval [0, 1] when there is low SNR.
    if filter_bad:
        timeseries[(timeseries>1) | (timeseries<0)] = np.nan
    return np.nanmean(timeseries, axis=-1)

def get_pixel_values(delta_values, region, filter_bad=False):
    mask, _ = region.to_mask_slice(delta_values)
    delta = delta_values.raw_data.T[mask.T].T

    if filter_bad:
        delta[(delta>0.1) | (delta<-0.1)] = np.nan
    return delta

gca = pat.GasChallengeAnalyser(display_output=False)
dso2, _, (baseline_so2, baseline_sigma_so2) = gca.run(so2, pa_so2)

dce = pat.DCEAnalyser(display_output=False)
dicg, _, [baseline_icg, baseline_sigma_icg] = dce.run(pa_dce.get_scan_unmixed(), pa_dce)

/home/cri.camres.org/else01/patato/patato/core/image_structures/reconstruction_image.py:43: UserWarning: Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.
  warnings.warn("Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.")


In [4]:
pa_so2.get_wavelengths()

array([700., 730., 750., 760., 770., 800., 820., 840., 850., 880.])

In [5]:
icg_df = pd.DataFrame({"times": np.concatenate([icg_times] * 3), 
                       "ICG": np.concatenate([get_trace(icg, r) for r in [roi_reference, roi_tumour_right, roi_tumour_left]]),
                       "Region": ["Spine"] * len(icg_times) + ["Right Tumour"] * len(icg_times) + ["Left Tumour"] * len(icg_times)
                      })


so2_df = pd.DataFrame({"times": np.concatenate([times] * 3), 
                       "SO2": np.concatenate([get_trace(so2, r, True) for r in [roi_reference, roi_tumour_right, roi_tumour_left]]),
                       "Region": ["Spine"] * len(times) + ["Right Tumour"] * len(times) + ["Left Tumour"] * len(times)
                      })

im_recon = rec.imshow()
im_dso2 = dso2.imshow(roi_mask=[roi_reference, roi_tumour_right, roi_tumour_left])
im_dicg = dicg.imshow(roi_mask=[roi_reference, roi_tumour_right, roi_tumour_left])
plt.close()

In [6]:
dso2_array = im_dso2.get_array().data
iqr = np.nanpercentile(dso2_array, 75) - np.nanpercentile(dso2_array, 25)
med = np.nanmedian(dso2_array)
dso2_array[dso2_array > med + 3 * iqr] = np.nan
dso2_array[dso2_array < med - 3 * iqr] = np.nan

dicg_array = im_dicg.get_array().data
iqr = np.nanpercentile(dicg_array, 75) - np.nanpercentile(dicg_array, 25)
med = np.nanmedian(dicg_array)
dicg_array[dicg_array > med + 6 * iqr] = np.nan
dicg_array[dicg_array < med - 6 * iqr] = np.nan

rec_array = im_recon.get_array().data

In [7]:
dicgs = [get_pixel_values(dicg, r) for r in [roi_reference, roi_tumour_right, roi_tumour_left]]
dso2s = [get_pixel_values(dso2, r, True) for r in [roi_reference, roi_tumour_right, roi_tumour_left]]

compare_df = pd.DataFrame({"dicg": np.concatenate(dicgs), 
                           "dso2": np.concatenate(dso2s),
                           "Region": sum([[s] * len(j) for s, j in zip(["Reference", "Right Tumour", "Left Tumour"], dso2s)], start=[])
                      })

compare_df

,dicg,dso2,Region
0,0.311121,0.060318,Reference
1,0.342048,0.066264,Reference
2,0.303935,0.055998,Reference
3,0.379399,0.078032,Reference
4,0.415269,0.081478,Reference
...,...,...,...
1950,0.248291,0.049637,Left Tumour
1951,0.159100,0.032974,Left Tumour
1952,0.190295,0.036675,Left Tumour
1953,0.216423,0.043448,Left Tumour


In [8]:
# Phantom figure:
import matplotlib
matplotlib.rc('font', family='Arial') 

p_clinical = get_msot_phantom_example("clinical")[4:5]
p_clinical.set_default_recon()
p_preclinical = get_msot_phantom_example("preclinical")
p_preclinical.set_default_recon()

  0%|          | 0/788003578 [00:00<?, ?it/s]

/home/cri.camres.org/else01/patato/patato/core/image_structures/reconstruction_image.py:43: UserWarning: Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.
  warnings.warn("Old version of PATATO used to generate this data. We recommend to re-run the image reconstruction.")


InvalidSchema: No connection adapters were found for 'hhttps://www.repository.cam.ac.uk/bitstream/handle/1810/345836/preclinical_phantom.hdf5?sequence=3&isAllowed=y'

In [ ]:
inch = 2.54
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13.5 / inch, 7.5 / inch))
r = p_clinical.get_scan_reconstructions()[:, 20:31]
ax1.set_title("Clinical Phantom", fontsize=8)
print(p_clinical.get_wavelengths()[20])
r.imshow(ax=ax1)

r = p_preclinical.get_scan_reconstructions()[:, 18:31]
print(p_preclinical.get_wavelengths()[18])
ax2.set_title("Preclinical Phantom", fontsize=8)
r.imshow(ax=ax2)
fig.subplots_adjust(left=0, right=1, bottom=0, hspace=0.005, wspace=0.05)
plt.savefig("test_figure.pdf")
plt.savefig("test_figure.png", dpi=300)
plt.show()

In [ ]:
p_preclinical.get_scan_datetime()